In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "src")))
from utils import Utils

In [ ]:
collects = ["20240305_002_Iver3069_ABS2",
            "20240306_001_Iver3069_ABS2",
            "20240307_001_Iver3069_ABS2",
            "20240307_002_Iver3069_ABS2",
            "20240307_003_Iver3069_ABS2",
            "20241112_001_Iver3069_ABS2",
            "20241216_001_Iver3069_ABS2",
            "20241218_001_Iver3069_ABS2",
            "20241218_002_Iver3069_ABS2"]

# all_meta = pd.read_csv(r"C:\Users\ageglio\ageglio-1\Indexing-AUV-data\2019-2023-20250110\header_2024_combined.csv", index_col=0, low_memory=False)
unpacked = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2024.csv", index_col=0, low_memory=False)
qto_metadata = unpacked[unpacked.collect_id.isin(collects)]
print(qto_metadata.shape)
qto_metadata_u = qto_metadata[qto_metadata.Usability == "Usable"]
qto_metadata_u.to_csv(r"inference\Mussel_control_quick_to\mussel_control_metadata.csv")
mcm_lst = list(qto_metadata_u.image_path)
filename = r"inference\Mussel_control_quick_to\mussel_control_metadata_lst.csv"
assert len(mcm_lst) == len(qto_metadata_u) 
Utils.write_list_txt(filename, mcm_lst)
len(mcm_lst)

In [ ]:
qto_metadata_u.collect_id.value_counts()

In [ ]:
GobyFinder_results = pd.read_csv(r"Z:\Proj_Yolo\Mussel_Control_2024\mussel_control_assmt_02122025.csv")
GobyFinder_results_p = GobyFinder_results[GobyFinder_results.conf_pass==1]
GobyFinder_results_u = GobyFinder_results.drop_duplicates(subset="Filename")
GobyFinder_results_u = GobyFinder_results_u[GobyFinder_results_u.conf_pass==0]
GobyFinder_results_up = pd.concat([GobyFinder_results_p, GobyFinder_results_u])
GobyFinder_results_up = GobyFinder_results_up.sort_values(by="Filename").reset_index(drop=True)
GobyFinder_results_up

In [ ]:
# List of file paths
alden_results = [
    r"Z:\Proj_SubstrateClassification\AldenResults\20240306_001_Iver3069_ABS2\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20240306_001_Iver3069_ABS2_NeedsRedo\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20240307_001_Iver3069_ABS2\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20240307_002_Iver3069_ABS2\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20240307_003_Iver3069_ABS2\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20241112_001_Iver3069_ABS2\Substrate.csv",
    r"Z:\Proj_SubstrateClassification\AldenResults\20241216_001_Iver3069_ABS2\Substrate.csv"
]

# Initialize an empty list to store dataframes
dfs = []
# Loop through the file paths and check if they exist
for file_path in alden_results:
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, usecols=["Filename", "2class_tight", "6class_tight", "9class_tight"])
        dfs.append(df)
# Concatenate all dataframes
alden_concatenated_df = pd.concat(dfs, ignore_index=True)

# List of file paths for the PDFs
dnn_substrate = [
    r"Z:\Proj_Yolo\Mussel_Control_2024\mussel_control_6c_substrate_predictions.csv",
    r"Z:\Proj_Yolo\Mussel_Control_2024\mussel_control_2c_substrate_predictions.csv"
]

# Read the PDFs into dataframes
df_6c = pd.read_csv(dnn_substrate[0], index_col=0)[["Filename", "substrate_class_6c"]]
df_2c = pd.read_csv(dnn_substrate[1], index_col=0)[["Filename", "substrate_class_2c"]]

# Assert they are the same length
assert len(df_6c) == len(df_2c), "Dataframes are not the same length"

# Merge the dataframes on the "Filename" column
merged_df = pd.merge(df_6c, df_2c, on="Filename", suffixes=('_6c', '_2c'))
merged_df = merged_df.rename(columns={"substrate_class_6c":"substrate_dnn_6c", "substrate_class_2c": "substrate_dnn_6c"})
len(merged_df)

mussel_Finder_results = pd.read_csv(r"Z:\Proj_Yolo\Mussel_Control_2024\mussel_control_mussel_finder_predictions.csv")[["Filename", "Coverage"]]

In [ ]:
GobyFinder_results_substrate = pd.merge(GobyFinder_results_up, alden_concatenated_df.drop_duplicates(subset="Filename"), on="Filename", how="left")
GobyFinder_results_substrate = pd.merge(GobyFinder_results_substrate, merged_df, on="Filename", how="left")
print(GobyFinder_results_substrate.shape)
GobyFinder_results_substrate = GobyFinder_results_substrate.drop(columns=["substrate_class"])
GobyFinder_results_substrate_cover = pd.merge(GobyFinder_results_substrate, mussel_Finder_results, on="Filename", how="left")
GobyFinder_results_substrate_cover.to_csv(r"Z:\Proj_Yolo\Mussel_Control_2024\GobyFinder_results_substrate_RF_DNN_Cover.csv")

In [ ]:
GobyFinder_results_substrate_cover["collect_id"].value_counts()

In [ ]:
GobyFinder_results_substrate_cover